In [ ]:
#Importación de librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
import numpy as np
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer

import seaborn as sns
import os
import shutil
import json
import joblib

!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [ ]:
#Configuración de la API de Kaggle
os.environ['KAGGLE_CONFIG_DIR']='/content/'

data = {"username":"kellyarangohenao","key":"49fe4a9967395e59af1b5df1a5194b48"}
with open('kaggle.json','w') as file:
  json.dump(data, file, indent=4)


!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.12GB/s]


In [ ]:
#Descompresión y validación de archivos
!unzip udea*.zip > /dev/null
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


In [ ]:
#Descarga y Exploración de Datos
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
copy_df = train_df.copy()

In [ ]:
train_df.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'coef_1', 'coef_2', 'coef_3', 'coef_4'],
      dtype='object')

In [ ]:
#Conversión de variables categóricas a numéricas
def rendimientoglobal(df):
  map_df = {
    "alto":3,
    "medio-alto":2,
    "medio-bajo":1,
    "bajo": 0
  }
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(map_df)

In [ ]:
def rendimientoglobalreverse(df):
    df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].round().astype(int)
    map_df = {
        3: "alto",
        2: "medio-alto",
        1: "medio-bajo",
        0: "bajo"
    }
    df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(map_df)
    return df

In [ ]:
def estrato(df):
    map_df = {
        'Estrato 1': 1,
        'Estrato 2': 3,
        'Estrato 3': 4,
        'Estrato 4': 5,
        'Estrato 5': 6,
        'Estrato 6': 7,
        'Sin Estrato': 2
    }

    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].map(map_df)

    # Calcula la moda
    modas = df['FAMI_ESTRATOVIVIENDA'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0


    def generar_valores(row):
        if pd.isna(row):
            return moda
        return row

    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].apply(generar_valores)


In [ ]:
import random
def internet(df):
  map_df = {
    'Si': 1,
    'No': 0,
  }
  df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].map(map_df)
  percent=np.round(df['FAMI_TIENEINTERNET'].mean(),4)

  def assign_value(row):
    est_value=row['FAMI_ESTRATOVIVIENDA']
    if pd.isnull(row['FAMI_TIENEINTERNET']):
        if est_value > 2:
            return 1
        elif np.round(random.random(),4) <= percent:
            return 1
        else:
            return 0
    else:
        return row['FAMI_TIENEINTERNET']

  df['FAMI_TIENEINTERNET'] = df.apply(assign_value, axis=1)


In [ ]:
def matriculapropia(df):


    res_mean = copy_df.groupby('ESTU_PAGOMATRICULAPROPIO')["RENDIMIENTO_GLOBAL"].mean()
    res_std = copy_df.groupby('ESTU_PAGOMATRICULAPROPIO')["RENDIMIENTO_GLOBAL"].std()


    df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].map(res_mean)


    def generar_valores_normales(row):
        if pd.isna(row):
            mean = res_mean.mean()
            std = res_std.mean()
            return np.random.normal(mean, std)
        return row


    df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].apply(generar_valores_normales)

In [ ]:
def horassemanatrabaja(df):
    map_df = {
        '0': 5,
        'Entre 11 y 20 horas': 3,
        'Entre 21 y 30 horas': 2,
        'Menos de 10 horas': 4,
        'Más de 30 horas': 1
    }

    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map(map_df)

    # calcula la moda
    modas = df['ESTU_HORASSEMANATRABAJA'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0


    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna(moda)

In [ ]:
def valormatriculauniversidad(df):
    map_df = {
        'Entre 1 millón y menos de 2.5 millones': 1,
        'Entre 2.5 millones y menos de 4 millones': 2,
        'Entre 4 millones y menos de 5.5 millones': 6,
        'Entre 5.5 millones y menos de 7 millones': 7,
        'Entre 500 mil y menos de 1 millón': 3,
        'Menos de 500 mil': 5,
        'Más de 7 millones': 8,
        'No pagó matrícula': 4
    }

    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(map_df)

    # calcula la moda
    modas = df['ESTU_VALORMATRICULAUNIVERSIDAD'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0


    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(moda)

In [ ]:
def educacionpadre(df):
    map_df = {
        'Educación profesional completa': 11,
        'Educación profesional incompleta': 10,
        'Ninguno': 1,
        'No Aplica': 7,
        'No sabe': 9,
        'Postgrado': 12,
        'Primaria completa': 3,
        'Primaria incompleta': 2,
        'Secundaria (Bachillerato) completa': 6,
        'Secundaria (Bachillerato) incompleta': 4,
        'Técnica o tecnológica completa': 8,
        'Técnica o tecnológica incompleta': 5
    }

    df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].map(map_df)

    modas = df['FAMI_EDUCACIONPADRE'].mode()
    moda = modas.iloc[0] if not modas.empty else 0

    df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna(moda)

In [ ]:
def educacionmadre(df):
    map_df = {
        'Educación profesional completa': 11,
        'Educación profesional incompleta': 10,
        'Ninguno': 1,
        'No Aplica': 4,
        'No sabe': 6,
        'Postgrado': 12,
        'Primaria completa': 3,
        'Primaria incompleta': 2,
        'Secundaria (Bachillerato) completa': 7,
        'Secundaria (Bachillerato) incompleta': 5,
        'Técnica o tecnológica completa': 9,
        'Técnica o tecnológica incompleta': 8
    }

    df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].map(map_df)

    # calcula la moda
    modas = df['FAMI_EDUCACIONMADRE'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0


    df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna(moda)

In [ ]:
def periodo(df):
    df['PERIODO'] = df['PERIODO'].astype(str)

    map_df = {
        "20183": 5,
        "20184": 1,
        "20194": 9,
        "20195": 3,
        "20196": 4,
        "20202": 8,
        "20203": 6,
        "20212": 2,
        "20213": 7
    }

    df['PERIODO'] = df['PERIODO'].map(map_df)

    modas = df['PERIODO'].mode()
    if not modas.empty:
        moda = modas.iloc[0]
    else:
        moda = 0

    df['PERIODO'] = df['PERIODO'].fillna(moda)


In [ ]:
def departamento(df):


    res_mean = copy_df.groupby('ESTU_PRGM_DEPARTAMENTO')["RENDIMIENTO_GLOBAL"].mean()


    df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].map(res_mean)


    def generar_valores_normales(row):
        if pd.isna(row):
            mean = copy_df['RENDIMIENTO_GLOBAL'].mean()
            return mean
        return row


    df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].apply(generar_valores_normales)

In [ ]:
def programaacademico(df):


    res_mean = copy_df.groupby('ESTU_PRGM_ACADEMICO')["RENDIMIENTO_GLOBAL"].mean()


    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].map(res_mean)


    def generar_valores_normales(row):
        if pd.isna(row):  # Si el valor es nulo
            mean = copy_df['RENDIMIENTO_GLOBAL'].mean()
            return mean
        return row


    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(generar_valores_normales)


In [ ]:
def columnas_binarias(df):
    """Convierte las columnas binarias'"""
    binarias = [
        'FAMI_TIENELAVADORA',
        'FAMI_TIENEAUTOMOVIL',
        'ESTU_PRIVADO_LIBERTAD',
        'FAMI_TIENECOMPUTADOR',
        'FAMI_TIENEINTERNET.1'
    ]
    for col in binarias:
        if col in df.columns:
            df[col] = df[col].map({'Si': 1, 'No': 0})
            df[col] = df[col].fillna(0)

In [ ]:
def limpiar_df(df, predict=False, test=False):
  if not predict:
    if not test:
      rendimientoglobal(df)
    estrato(df)
    internet(df)
    matriculapropia(df)
    valormatriculauniversidad(df)
    educacionpadre(df)
    educacionmadre(df)
    horassemanatrabaja(df)
    departamento(df)
    programaacademico(df)
    periodo(df)
    columnas_binarias(df)
    return df
  elif predict:
    rendimientoglobal(df)

In [ ]:
limpiar_df(copy_df, predict=True)
traindf = limpiar_df(train_df)
testdf = limpiar_df(test_df, test=True)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time

df = train_df.copy()

In [ ]:
#Separación de variables predictoras y objetivo

X = df.drop(columns=["RENDIMIENTO_GLOBAL","ID"], axis=1)
y = df["RENDIMIENTO_GLOBAL"]

In [ ]:
#División del Conjunto de Datos en Entrenamiento y Validación

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


X_train = X_train.fillna(0)
X_test = X_test.fillna(0)


base_estimator = DecisionTreeClassifier(max_depth=5)
classifier = AdaBoostClassifier(estimator=base_estimator,n_estimators=50,learning_rate=0.3,random_state=73)

start = time.time()

In [ ]:



# Entrenar el modelo
classifier.fit(X_train, y_train)

# Hacer predicciones
y_pred = classifier.predict(X_test)

# Evaluar
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión: {accuracy}')



Precisión: 0.41662093862815885
Entrenamiento completado en 156.03 segundos.


In [ ]:
#Predicción
df_test = testdf.copy()


ids = df_test["ID"].copy()


df_test.drop(columns=["ID", "Unnamed: 0"], axis=1, inplace=True, errors='ignore')


predicciones = classifier.predict(df_test)


submission = pd.DataFrame({
    "ID": ids,
    "RENDIMIENTO_GLOBAL": predicciones
})


if 'rendimientoglobalreverse' in globals():
    submission = rendimientoglobalreverse(submission)


y.to_csv("submission.csv", index=False)

print(y.head())

# Guardar el archivo en formato CSV
submission.to_csv("submission.csv", index=False)
print(" Archivo submission.csv guardado correctamente.")

0    2
1    0
2    0
3    3
4    1
Name: RENDIMIENTO_GLOBAL, dtype: int64
✅ Archivo submission.csv guardado correctamente.


In [ ]:
#Envío del Archivo de Predicciones a la Competencia de Kaggle
!kaggle competitions submit -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia -f submission.csv -m "Kelly Julieth Arango Henao Random Forest"



100% 4.10M/4.10M [00:00<00:00, 9.00MB/s]
Successfully submitted to UDEA/ai4eng 20251 - Pruebas Saber Pro Colombia